In [1]:
import numpy as np
import pandas as pd

# Trades data

In [2]:
df = pd.read_csv("data/bayc.csv", index_col=0)

## clean up price and datetime columns

In [3]:
# rename price
df.rename(columns={'amount_original':'trade_price'}, inplace=True)

# drop if price missing
df = df[df['trade_price']!='<nil>']

# convert types
df['trade_price'] = df['trade_price'].astype(float)
df['block_time'] = pd.to_datetime(df['block_time'])
df['block_date'] = pd.to_datetime(df['block_date'])
df['block_month'] = pd.to_datetime(df['block_month'])

# drop observation if trade_price above max_price or below min_price
df = df[(df['trade_price'] <= df['price_max_eth']) & (df['trade_price'] >= df['price_min_eth'])]

## add variables: last_trade_price, last_trade_timediff

In [4]:
# note: dataset is pre-sorted by token_id and block_time
df['last_trade_price'] = df.groupby('token_id')['trade_price'].shift(1)
df['last_trade_time'] = df.groupby('token_id')['block_time'].shift(1)
df['last_trade_timediff'] = df['block_time'] - df['last_trade_time']
df['last_trade_timediff'] = df['last_trade_timediff'].dt.total_seconds() / 86400

## creating lagged columns for market price/volume data

In [5]:
# check if dates are consecutive
dates = df['block_date'].sort_values().unique()
dates_diff = dates[1:] - dates[:-1]
sum(dates_diff != pd.Timedelta('1 days'))

0

In [6]:
# Aggregating the data by block_date
lag_columns = ['volume_eth', 'price_p5_eth', 'price_max_eth', 'price_min_eth']
daily_aggregated = df.groupby('block_date')[lag_columns].mean().reset_index()

# Creating lagged columns
for col in lag_columns:
    daily_aggregated[col + '_lag1'] = daily_aggregated[col].shift(1)

daily_aggregated.head()

,block_date,volume_eth,price_p5_eth,price_max_eth,price_min_eth,volume_eth_lag1,price_p5_eth_lag1,price_max_eth_lag1,price_min_eth_lag1
0,2021-04-30,0.160000,0.160000,0.16,0.16000,NaN,NaN,NaN,NaN
1,2021-05-01,560.845702,0.092417,10.00,0.04000,0.160000,0.160000,0.16,0.16000
2,2021-05-02,1330.337944,0.240629,21.00,0.13965,560.845702,0.092417,10.00,0.04000
3,2021-05-03,960.840859,0.365829,15.00,0.04500,1330.337944,0.240629,21.00,0.13965
4,2021-05-04,295.019333,0.300000,8.00,0.10000,960.840859,0.365829,15.00,0.04500


In [7]:
df = df.merge(
    daily_aggregated[['block_date', 'volume_eth_lag1', 'price_p5_eth_lag1', 'price_max_eth_lag1', 'price_min_eth_lag1']], 
    on='block_date', how='left')
df = df[df['block_date'] != '2021-04-30']

## create log price variable as alternative outcome

In [8]:
df['trade_price_log'] = np.log(df['trade_price'])

# Traits/Rarity data

In [9]:
traits_df = pd.read_csv("data/bayc_traits.csv", index_col=0)
traits_df.head()

,token_id,rarity_rank,Background_value,Background_count,Hat_value,Hat_count,Mouth_value,Mouth_count,Eyes_value,Eyes_count,Fur_value,Fur_count,Clothes_value,Clothes_count,Earring_value,Earring_count
0,5,3447,Army Green,1243,Bayc Flipped Brim,231.0,Dumbfounded,505,X Eyes,243,Brown,1370,Bayc T Red,140.0,NaN,NaN
1,6,5599,Yellow,1283,S&m Hat,235.0,Bored,2272,Crazy,407,Cream,636,Tweed Suit,141.0,NaN,NaN
2,8,5267,Aquamarine,1266,Beanie,578.0,Bored,2272,Robot,350,Zombie,302,NaN,NaN,Gold Stud,439.0
3,9,2389,Purple,1291,Seaman's Hat,420.0,Small Grin,272,Sleepy,751,Brown,1370,Stunt Jacket,178.0,Silver Stud,823.0
4,10,3735,Aquamarine,1266,Bayc Hat Red,119.0,Bored,2272,Eyepatch,333,Dmt,215,Navy Striped Tee,334.0,NaN,NaN


## format optional traits
if trait is missing, put value = NA and count = # of items without the trait

In [10]:
optional_traits = {} # pairs of "trait" : "empty count"

for i,v in traits_df.isna().sum().items():
    if v>0: # if there are empty values in the trait
        trait = i.split("_")[0]
        optional_traits[trait] = v
print(optional_traits)

{'Hat': 2042, 'Clothes': 1690, 'Earring': 6383}


In [11]:
for trait, count in optional_traits.items():
    traits_df[trait+"_value"] = traits_df[trait+"_value"].fillna("Empty")
    traits_df[trait+"_count"] = traits_df[trait+"_count"].fillna(count).astype(int)

In [12]:
cleaned = df.merge(traits_df, on='token_id', how='left')
cleaned.to_csv("data/bayc_cleaned.csv")